In [3]:
import pandas as pd
import xml.etree.ElementTree as ET

# Especifica la ruta al archivo Excel
archivo_excel = 'extract_glosa.xlsx'

# Lee el archivo Excel en un DataFrame de Pandas
df = pd.read_excel(archivo_excel)

id, rut_emisor, folio, numero de items, glosa, monto.

In [4]:
# filas = []

# i = 0

# for xml in df['convert_from'].head(20):

#     root = ET.fromstring(xml)

#     # Utiliza el namespace de XML
#     namespace = {'default': 'http://www.sii.cl/SiiDte'}

#     rut_emisor = root.find('.//default:RutEmisor', namespace).text

#     razon_social = root.find('.//{http://www.sii.cl/SiiDte}RznSoc').text

#     NmbItem = root.find('.//{http://www.sii.cl/SiiDte}NmbItem').text

#     NmbItem = root.find('.//{http://www.sii.cl/SiiDte}NmbItem').text

#     # a veces DscItem no existe
#     DscItem = root.find('.//{http://www.sii.cl/SiiDte}DscItem')

#     if DscItem is not None: # Crea nuevas filas para el df con DscItem
#         nuevaFila = {
#             "rut_emisor": rut_emisor,
#             "razon_social": razon_social,
#             "NmbItem": NmbItem,
#             "DscItem": root.find('.//{http://www.sii.cl/SiiDte}DscItem').text
#         }
#         filas.append(nuevaFila)
#     else: # Crea nuevas filas para el df sin DscItem
#         nuevaFila = {
#             "rut_emisor": rut_emisor,
#             "razon_social": razon_social,
#             "NmbItem": NmbItem,
#             "DscItem": None
#         }
#         filas.append(nuevaFila)

Pasos acceder a los valores del detalle

- tener en cuenta que hay mas de un detalle

In [38]:
ns = {'ns': 'http://www.sii.cl/SiiDte'}

filas = []


def get_element_value(element, tag):
    return element.find(f'ns:{tag}', ns).text if element.find(f'ns:{tag}', ns) is not None else None
i=0
for xml in df['convert_from'].head(4):
    root = ET.fromstring(xml)
    # if(i>2):
        # Utiliza el namespace de XML
    namespace = {'default': 'http://www.sii.cl/SiiDte'}

    Folio = root.find('.//{http://www.sii.cl/SiiDte}Folio').text
    RutEmisor = root.find('.//{http://www.sii.cl/SiiDte}RutEmisor').text
    
    print("Folio:", Folio)
    print("RutEmisor:", RutEmisor)


    detalles = root.findall('.//ns:Detalle', ns)
    for detalle in detalles:
        NroLinDet = get_element_value(detalle, 'NroLinDet')
        NmbItem = get_element_value(detalle, 'NmbItem')
        MontoItem = get_element_value(detalle, 'MontoItem')
        DscItem = get_element_value(detalle, 'DscItem')
        print(f"Detalle {NroLinDet}:")
        print("  NmbItem:", NmbItem)
        print("  MontoItem:", MontoItem)
        print("  DscItem:", DscItem)
        if DscItem is not None:
            nuevaFila = {
            "RutEmisor": RutEmisor,
            "Folio": Folio,
            "N_Item":NroLinDet,
            "DscItem": DscItem,
            "NmbItem": NmbItem,
            "MontoItem": MontoItem,
            
            }
            filas.append(nuevaFila)
        else:
            nuevaFila = {
            "RutEmisor": RutEmisor,
            "Folio": Folio,
            "N_Item":NroLinDet,
            "DscItem": None,
            "NmbItem": NmbItem,
            "MontoItem": MontoItem,
            }
            filas.append(nuevaFila)
    i=i+1

    print('------------',i)

df_new = pd.DataFrame(filas)

Folio: 440
RutEmisor: 77566800-8
Detalle 1:
  NmbItem: Arriendo local 2 Puerto
  MontoItem: 5156309
  DscItem: montt    valor proporcional mes de Enero uf 140
------------ 1
Folio: 89
RutEmisor: 16383874-5
Detalle 1:
  NmbItem: Arriendo Álvaro Casanova
  MontoItem: 898498
  DscItem: None
------------ 2
Folio: 937
RutEmisor: 76789922-K
Detalle 1:
  NmbItem: Usage: Remote Hands 1 Day
  MontoItem: 916160
  DscItem: Ticket: 822574: 12/19/23: Requested by: carlos.meza@mundopacifico.cl SOF ID: RVRH-EDCSCL01-PACIFICO Periodo de facturación: 01/01/2024-01/31/2024  Total USD: $1.190,00  Valor USD: 916,16 USD al 29.01.2024  Invoice: 1-194-452
------------ 3
Folio: 50872845
RutEmisor: 96689310-9
Detalle 1:
  NmbItem: COMISION POR VENTAS NACIONALES. EMISOR R.U.T.: 97951000-4
  MontoItem: 114
  DscItem: None
Detalle 2:
  NmbItem: SERVICIO TBK 12/2023
  MontoItem: 10807
  DscItem: None
Detalle 3:
  NmbItem: COMISIONES POR VENTAS NACIONALES
  MontoItem: 2922
  DscItem: None
------------ 4


In [37]:
df_new.head(10)

,RutEmisor,Folio,N_Item,DscItem,NmbItem,MontoItem
0,77566800-8,440,1,montt valor proporcional mes de Enero uf 140,Arriendo local 2 Puerto,5156309
1,16383874-5,89,1,None,Arriendo Álvaro Casanova,898498
2,76789922-K,937,1,Ticket: 822574: 12/19/23: Requested by: carlos...,Usage: Remote Hands 1 Day,916160
3,96689310-9,50872845,1,None,COMISION POR VENTAS NACIONALES. EMISOR R.U.T.:...,114
4,96689310-9,50872845,2,None,SERVICIO TBK 12/2023,10807
5,96689310-9,50872845,3,None,COMISIONES POR VENTAS NACIONALES,2922


In [41]:
def Combinar_columnas(df, col1, col2, nueva_columna):
    """
    Combina dos columnas de un DataFrame de pandas por índice y crea una nueva columna con el resultado.

    :param df: DataFrame de pandas.
    :param col1: Nombre de la primera columna a combinar.
    :param col2: Nombre de la segunda columna a combinar.
    :param nueva_columna: Nombre de la nueva columna que contendrá la combinación.
    :return: DataFrame con la nueva columna añadida.
    """
    df[nueva_columna] = df[col1].astype(str) + df[col2].astype(str)
    df = df.drop(columns=[col1, col2])
    return df

df_new2 = Combinar_columnas(df_new, 'DscItem', 'NmbItem', 'combinada')

In [47]:
ns = {'ns': 'http://www.sii.cl/SiiDte'}

# filas = []

filas_sin = []
filas_con = []

def get_element_value(element, tag):
    return element.find(f'ns:{tag}', ns).text if element.find(f'ns:{tag}', ns) is not None else None
i=0
for xml in df['convert_from'].head(4):
    root = ET.fromstring(xml)
    # if(i>2):
        # Utiliza el namespace de XML
    namespace = {'default': 'http://www.sii.cl/SiiDte'}

    Folio = root.find('.//{http://www.sii.cl/SiiDte}Folio').text
    RutEmisor = root.find('.//{http://www.sii.cl/SiiDte}RutEmisor').text
    
    print("Folio:", Folio)
    print("RutEmisor:", RutEmisor)


    detalles = root.findall('.//ns:Detalle', ns)
    for detalle in detalles:
        NroLinDet = get_element_value(detalle, 'NroLinDet')
        NmbItem = get_element_value(detalle, 'NmbItem')
        MontoItem = get_element_value(detalle, 'MontoItem')
        DscItem = get_element_value(detalle, 'DscItem')
        print(f"Detalle {NroLinDet}:")
        print("  NmbItem:", NmbItem)
        print("  MontoItem:", MontoItem)
        print("  DscItem:", DscItem)
        if DscItem is not None:
            nuevaFila = {
            "RutEmisor": RutEmisor,
            "Folio": Folio,
            "N_Item":NroLinDet,
            "NmbItem": NmbItem,
            "DscItem": DscItem,
            "MontoItem": MontoItem,
            
            }
            filas_con.append(nuevaFila)
        else:
            nuevaFila = {
            "RutEmisor": RutEmisor,
            "Folio": Folio,
            "N_Item":NroLinDet,
            "NmbItem": NmbItem,
            "DscItem": None,
            "MontoItem": MontoItem,
            }
            filas_sin.append(nuevaFila)
    i=i+1

    print('------------',i)

# agregar las nuevas filas a los distintos df
df_con = pd.DataFrame(filas_con) # df con DscItem
df_sin = pd.DataFrame(filas_sin) # df sin DscItem

Folio: 440
RutEmisor: 77566800-8
Detalle 1:
  NmbItem: Arriendo local 2 Puerto
  MontoItem: 5156309
  DscItem: montt    valor proporcional mes de Enero uf 140
------------ 1
Folio: 89
RutEmisor: 16383874-5
Detalle 1:
  NmbItem: Arriendo Álvaro Casanova
  MontoItem: 898498
  DscItem: None
------------ 2
Folio: 937
RutEmisor: 76789922-K
Detalle 1:
  NmbItem: Usage: Remote Hands 1 Day
  MontoItem: 916160
  DscItem: Ticket: 822574: 12/19/23: Requested by: carlos.meza@mundopacifico.cl SOF ID: RVRH-EDCSCL01-PACIFICO Periodo de facturación: 01/01/2024-01/31/2024  Total USD: $1.190,00  Valor USD: 916,16 USD al 29.01.2024  Invoice: 1-194-452
------------ 3
Folio: 50872845
RutEmisor: 96689310-9
Detalle 1:
  NmbItem: COMISION POR VENTAS NACIONALES. EMISOR R.U.T.: 97951000-4
  MontoItem: 114
  DscItem: None
Detalle 2:
  NmbItem: SERVICIO TBK 12/2023
  MontoItem: 10807
  DscItem: None
Detalle 3:
  NmbItem: COMISIONES POR VENTAS NACIONALES
  MontoItem: 2922
  DscItem: None
------------ 4


In [48]:
df_con.head()

,RutEmisor,Folio,N_Item,NmbItem,DscItem,MontoItem
0,77566800-8,440,1,Arriendo local 2 Puerto,montt valor proporcional mes de Enero uf 140,5156309
1,76789922-K,937,1,Usage: Remote Hands 1 Day,Ticket: 822574: 12/19/23: Requested by: carlos...,916160


In [49]:
df_sin.head(10)

,RutEmisor,Folio,N_Item,NmbItem,DscItem,MontoItem
0,16383874-5,89,1,Arriendo Álvaro Casanova,None,898498
1,96689310-9,50872845,1,COMISION POR VENTAS NACIONALES. EMISOR R.U.T.:...,None,114
2,96689310-9,50872845,2,SERVICIO TBK 12/2023,None,10807
3,96689310-9,50872845,3,COMISIONES POR VENTAS NACIONALES,None,2922


In [56]:
# Elimino la columna DscItem
df_sin = df_sin.drop(columns=['DscItem'])
df_sin = df_sin.rename(columns={'NmbItem': 'Glosa'})

In [ ]:
df_sin.head()

# Unir los df 

In [55]:
def eliminar_columna(df, columna):
    """
    Elimina una columna específica de un DataFrame.

    :param df: DataFrame del cual se quiere eliminar la columna.
    :param columna: Nombre de la columna que se desea eliminar.
    :return: DataFrame sin la columna especificada.
    """
    if columna in df.columns:
        df = df.drop(columns=[columna])
        print(f"Columna '{columna}' eliminada.")
    else:
        print(f"La columna '{columna}' no existe en el DataFrame.")
    return df

def renombrar_columna(df, nombre_antiguo, nombre_nuevo):
    """
    Renombra una columna específica de un DataFrame.

    :param df: DataFrame en el cual se quiere renombrar la columna.
    :param nombre_antiguo: Nombre actual de la columna que se desea renombrar.
    :param nombre_nuevo: Nuevo nombre para la columna.
    :return: DataFrame con la columna renombrada.
    """
    if nombre_antiguo in df.columns:
        df = df.rename(columns={nombre_antiguo: nombre_nuevo})
        print(f"Columna '{nombre_antiguo}' renombrada a '{nombre_nuevo}'.")
    else:
        print(f"La columna '{nombre_antiguo}' no existe en el DataFrame.")
    return df